In [105]:
import pandas as pd

expedition_data = pd.read_csv("../datasets/EXPEDITION_DATA.csv", encoding='cp1252')
peaks_data = pd.read_csv("../datasets/PEAKS_DATA.csv")

merged_data = pd.merge(left=expedition_data, right=peaks_data, how='inner', on='PEAKID')

In [106]:
merged_data.loc[:, 'TOTALDEATHS'] = merged_data['MDEATHS'] + merged_data['HDEATHS']
merged_data.loc[:, 'SUCCESS'] =  merged_data['SUCCESS1'].astype(bool) | merged_data['SUCCESS2'].astype(bool)

In [107]:
df = merged_data[['PEAKID', 'EXPID', 'YEAR', 'SEASON', 'SEASON_FACTOR', 'HOST', 'HOST_FACTOR',
                  'ROUTE1', 'ROUTE2', 'ROUTE3', 'ROUTE4', 'TOTMEMBERS', 'TERMREASON_FACTOR', 'O2USED', 'O2NONE', 'NOHIRED',
                  'TOTALDEATHS', 'PKNAME', 'PKNAME2', 'HEIGHTM', 'HEIGHTF', 'HIMAL_FACTOR',
                  'REGION_FACTOR', 'OPEN', 'PYEAR', 'PSTATUS_FACTOR', 'PSMTDATE']]

In [108]:
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

features = ['PKNAME', 'O2USED', 'SEASON']
target = 'SUCCESS'

X = df[features]
y = merged_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

encoder = TargetEncoder()
X_train_encoded = encoder.fit_transform(X_train, y_train)
X_test_encoded = encoder.transform(X_test)

model = LogisticRegression()
model.fit(X_train_encoded, y_train)

y_pred = model.predict(X_test_encoded)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.7231638418079096
